In [1]:
from pyspark.sql import SparkSession

24/12/31 14:13:51 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
spark = SparkSession.builder.appName("IMDB_Analysis").getOrCreate()

24/12/31 14:14:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
data_path = "/home/yashanand7227/IMDB_Dataset"
title_basics_path = data_path + "/title.basics.tsv.gz"
title_ratings_path = data_path + "/title.ratings.tsv.gz"
title_crew_path = data_path + "/title.crew.tsv.gz"
title_principals_path = data_path + "/title.principals.tsv.gz"


In [8]:
title_basics = spark.read.csv(title_basics_path, sep="\t", header=True, inferSchema=True)
title_ratings = spark.read.csv(title_ratings_path, sep="\t", header=True, inferSchema=True)
title_crew = spark.read.csv(title_crew_path, sep="\t", header=True, inferSchema=True)
title_principals = spark.read.csv(title_principals_path, sep="\t", header=True, inferSchema=True)




In [9]:
title_basics.show()
title_ratings.show()
title_crew.show()
title_principals.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|              

In [6]:
title_basics.count()

11338004

In [12]:
from pyspark.sql.functions import col
title_basics = title_basics.replace("\\N", None)

In [13]:
title_basics.show(10)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|   NULL|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|   NULL|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|   NULL|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|   NULL|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|   NULL|             1|              

In [14]:
    title_basics = title_basics.dropna(subset=["titleType", "startYear"])

In [15]:
title_basics.count()

9919523

In [22]:
title_types = title_basics.select("titleType").distinct()
is_adult_types = title_basics.select("isAdult").distinct()
title_types.show()
is_adult_types.show()

+------------+
|   titleType|
+------------+
|    tvSeries|
|tvMiniSeries|
|     tvMovie|
|     tvPilot|
|   tvEpisode|
|       movie|
|   tvSpecial|
|       video|
|   videoGame|
|     tvShort|
|       short|
+------------+



+-------+
|isAdult|
+-------+
|      0|
|      1|
+-------+



In [17]:
tv_movies = title_basics.filter(col("titleType")=="tvMovie")

In [18]:
tv_movies.count()

145244

In [19]:
tv_movies.show(10)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|       genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-------------+
|tt0030298|  tvMovie|       Julius Caesar|       Julius Caesar|      0|     1938|   NULL|           101|Drama,History|
|tt0038309|  tvMovie|      As You Like It|      As You Like It|      0|     1946|   NULL|          NULL|        Drama|
|tt0038738|  tvMovie|A Midsummer Night...|A Midsummer Night...|      0|     1946|   NULL|           150|Drama,Fantasy|
|tt0039445|  tvMovie|       Hamlet Part 1|       Hamlet Part 1|      0|     1947|   NULL|            88|        Drama|
|tt0039618|  tvMovie|The Merchant of V...|The Merchant of V...|      0|     1947|   NULL|            90|        Drama|
|tt0039624|  tvMovie|A Midsummer Night...|A Mids

In [20]:
title_type_counts = title_basics.groupBy("titleType").count()

In [21]:
title_type_counts.show()

+------------+-------+
|   titleType|  count|
+------------+-------+
|    tvSeries| 252102|
|tvMiniSeries|  54625|
|     tvMovie| 145244|
|     tvPilot|      1|
|   tvEpisode|7468624|
|       movie| 599739|
|   tvSpecial|  50349|
|       video| 301371|
|   videoGame|  40292|
|     tvShort|  10391|
|       short| 996785|
+------------+-------+



In [25]:
from pyspark.sql.functions import min
earliest_year = title_basics.select(min("startYear"))
earliest_year.show()

+--------------+
|min(startYear)|
+--------------+
|          1874|
+--------------+



In [27]:
title_basics = title_basics.filter(col("startYear").isNotNull())
title_basics.count()

9919523

In [36]:
title_basics = title_basics.dropDuplicates(["tconst"])
title_basics.count()

24/12/31 15:50:39 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/31 15:50:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/31 15:50:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/31 15:50:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/31 15:50:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/31 15:50:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/31 15:50:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/31 15:50:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/31 15:50:57 WARN RowBasedKeyValueBatch: Calling spill() on

9919523

In [37]:
title_ratings.count()

1517433

In [38]:
title_ratings = title_ratings.dropDuplicates(["tconst"])
title_ratings.count()

1517433

In [28]:
movies = title_basics.filter(col("titleType")=='movie')

In [29]:
movies.count()

599739

In [30]:
movies = movies.select("tconst", "primaryTitle", "isAdult", "startyear", "runtimeMinutes", "genres")

In [35]:
movies.count()

599739

In [32]:
movies_with_ratings = movies.join(title_ratings, on='tconst', how='inner')

In [33]:
movies_with_ratings.show()

+---------+--------------------+-------+---------+--------------+--------------------+-------------+--------+
|   tconst|        primaryTitle|isAdult|startyear|runtimeMinutes|              genres|averageRating|numVotes|
+---------+--------------------+-------+---------+--------------+--------------------+-------------+--------+
|tt0000009|          Miss Jerry|      0|     1894|            45|             Romance|          5.4|     216|
|tt0000147|The Corbett-Fitzs...|      0|     1897|           100|Documentary,News,...|          5.2|     544|
|tt0000502|            Bohemios|      0|     1905|           100|                NULL|          4.4|      18|
|tt0000574|The Story of the ...|      0|     1906|            70|Action,Adventure,...|          6.0|     952|
|tt0000591|    The Prodigal Son|      0|     1907|            90|               Drama|          5.7|      29|
|tt0000615|  Robbery Under Arms|      0|     1907|          NULL|               Drama|          4.3|      27|
|tt0000630

In [34]:
movies_with_ratings.count()

322777

Top 10 Highest Rated Movies


In [40]:
movies_with_ratings.orderBy(col("numVotes").desc()).show(10)

+---------+--------------------+-------+---------+--------------+--------------------+-------------+--------+
|   tconst|        primaryTitle|isAdult|startyear|runtimeMinutes|              genres|averageRating|numVotes|
+---------+--------------------+-------+---------+--------------+--------------------+-------------+--------+
|tt0111161|The Shawshank Red...|      0|     1994|           142|               Drama|          9.3| 2983880|
|tt0468569|     The Dark Knight|      0|     2008|           152|  Action,Crime,Drama|          9.0| 2964230|
|tt1375666|           Inception|      0|     2010|           148|Action,Adventure,...|          8.8| 2631192|
|tt0137523|          Fight Club|      0|     1999|           139|               Drama|          8.8| 2411870|
|tt0109830|        Forrest Gump|      0|     1994|           142|       Drama,Romance|          8.8| 2334889|
|tt0110912|        Pulp Fiction|      0|     1994|           154|         Crime,Drama|          8.9| 2289900|
|tt0816692

Top 10 Lowest Rated Movies

In [42]:
movies_with_ratings.orderBy([col("averageRating"), col("numVotes")], ascending=[True, False]).show(10)

+----------+--------------------+-------+---------+--------------+--------------------+-------------+--------+
|    tconst|        primaryTitle|isAdult|startyear|runtimeMinutes|              genres|averageRating|numVotes|
+----------+--------------------+-------+---------+--------------+--------------------+-------------+--------+
| tt5988370|                Reis|      0|     2017|           108|     Biography,Drama|          1.0|   74202|
| tt7221896|        Cumali Ceber|      0|     2017|           100|              Comedy|          1.0|   39499|
|tt13423846|          321 Action|      0|     2020|           100|               Drama|          1.0|   10225|
|tt13788842|2025 - The World ...|      0|     2021|            92|    Adventure,Sci-Fi|          1.0|    2638|
| tt1753912|Holnap történt - ...|      0|     2009|            82|Comedy,Crime,Mystery|          1.0|     976|
|tt15443254|    Cumali Ceber 666|      0|     2022|            82|              Comedy|          1.0|     910|
|

Most Popular Genres

In [45]:
from pyspark.sql.functions import explode, split

In [52]:
movies_split = movies_with_ratings.withColumn("genre_array", split(col("genres"),","))
movies_exploded = movies_split.withColumn("genre", explode(col("genre_array")))
movies_exploded.groupBy(col("genre")).count().orderBy(col("count").desc()).show(10)

+-----------+------+
|      genre| count|
+-----------+------+
|      Drama|144041|
|     Comedy| 78063|
|Documentary| 51294|
|    Romance| 35516|
|     Action| 32857|
|      Crime| 27841|
|   Thriller| 27112|
|     Horror| 24466|
|  Adventure| 19550|
|    Mystery| 12808|
+-----------+------+
only showing top 10 rows



Number of Movies in each year

In [61]:
movies_with_ratings.groupBy("startYear") \
    .count() \
    .withColumnRenamed("startYear", "Year") \
    .withColumnRenamed("count", "Num_of_movies") \
    .orderBy(col("Num_of_movies").desc()) \
    .show(10)


+----+-------------+
|Year|Num_of_movies|
+----+-------------+
|2022|        10983|
|2019|        10819|
|2023|        10736|
|2018|        10719|
|2017|        10567|
|2016|        10051|
|2015|         9649|
|2021|         9426|
|2014|         9400|
|2020|         8776|
+----+-------------+
only showing top 10 rows



Highest Rated Film on IMDB 2023

In [62]:
movies_with_ratings.write.parquet("movies_with_ratings.parquet")


In [63]:
spark.stop()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
                    .appName("ResumeAnalysis") \
                    .getOrCreate()

25/01/17 11:21:21 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
movies_with_ratings = spark.read.parquet("movies_with_ratings.parquet")

In [5]:
movies_with_ratings.show(10)

+---------+--------------------+-------+---------+--------------+--------------------+-------------+--------+
|   tconst|        primaryTitle|isAdult|startyear|runtimeMinutes|              genres|averageRating|numVotes|
+---------+--------------------+-------+---------+--------------+--------------------+-------------+--------+
|tt0000009|          Miss Jerry|      0|     1894|            45|             Romance|          5.4|     216|
|tt0000147|The Corbett-Fitzs...|      0|     1897|           100|Documentary,News,...|          5.2|     544|
|tt0000502|            Bohemios|      0|     1905|           100|                NULL|          4.4|      18|
|tt0000574|The Story of the ...|      0|     1906|            70|Action,Adventure,...|          6.0|     952|
|tt0000591|    The Prodigal Son|      0|     1907|            90|               Drama|          5.7|      29|
|tt0000615|  Robbery Under Arms|      0|     1907|          NULL|               Drama|          4.3|      27|
|tt0000630

In [6]:
movies_with_ratings.groupBy("startyear", "genres").count().orderBy("startyear").show()

+---------+--------------------+-----+
|startyear|              genres|count|
+---------+--------------------+-----+
|     1894|             Romance|    1|
|     1896|         Documentary|    1|
|     1897|Documentary,News,...|    2|
|     1898|         Documentary|    1|
|     1899|Documentary,News,...|    2|
|     1900|    Documentary,News|    1|
|     1900|         Documentary|    1|
|     1901|   Documentary,Sport|    1|
|     1901|         Documentary|    1|
|     1902|         Documentary|    1|
|     1903|     Biography,Drama|    1|
|     1903|         Documentary|    1|
|     1904|         Documentary|    1|
|     1905|               Music|    1|
|     1905|                NULL|    4|
|     1905|             Fantasy|    1|
|     1905|         Documentary|    1|
|     1905|               Drama|    1|
|     1906|Documentary,News,...|    1|
|     1906|                NULL|    3|
+---------+--------------------+-----+
only showing top 20 rows



In [8]:
from pyspark.sql.functions import col

movies_with_ratings.groupBy("genres").count().orderBy(col("count").desc()).show()


+--------------------+-----+
|              genres|count|
+--------------------+-----+
|               Drama|54897|
|         Documentary|34736|
|              Comedy|26132|
|                NULL|10796|
|        Comedy,Drama|10291|
|       Drama,Romance| 9842|
|              Horror| 7818|
|      Comedy,Romance| 5778|
|            Thriller| 4987|
|Comedy,Drama,Romance| 4852|
|              Action| 4594|
|         Crime,Drama| 4117|
|      Drama,Thriller| 3358|
|  Action,Crime,Drama| 3306|
|             Romance| 3069|
|        Action,Drama| 2760|
|               Adult| 2648|
|     Horror,Thriller| 2600|
|             Western| 2505|
|           Drama,War| 2148|
+--------------------+-----+
only showing top 20 rows



In [12]:
from pyspark.sql.functions import split, explode
movies_with_ratings = movies_with_ratings.drop("genres_array")


Creating a Start Schema with Dimension tables and Fact tables.

In [13]:
movies_with_ratings.show(10)

+---------+--------------------+-------+---------+--------------+--------------------+-------------+--------+
|   tconst|        primaryTitle|isAdult|startyear|runtimeMinutes|              genres|averageRating|numVotes|
+---------+--------------------+-------+---------+--------------+--------------------+-------------+--------+
|tt0000009|          Miss Jerry|      0|     1894|            45|             Romance|          5.4|     216|
|tt0000147|The Corbett-Fitzs...|      0|     1897|           100|Documentary,News,...|          5.2|     544|
|tt0000502|            Bohemios|      0|     1905|           100|                NULL|          4.4|      18|
|tt0000574|The Story of the ...|      0|     1906|            70|Action,Adventure,...|          6.0|     952|
|tt0000591|    The Prodigal Son|      0|     1907|            90|               Drama|          5.7|      29|
|tt0000615|  Robbery Under Arms|      0|     1907|          NULL|               Drama|          4.3|      27|
|tt0000630

In [15]:
from pyspark.sql.functions import monotonically_increasing_id
genres_exploded = movies_with_ratings.withColumn("genres_array", split(col("genres"), ","))
genres_exploded = genres_exploded.withColumn("genre", explode(col("genres_array")))


In [17]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window_spec = Window.orderBy("genre")
dim_genres = genres_exploded.select("genre").distinct().withColumn("genre_id", row_number().over(window_spec))
dim_genres.show(5)

25/01/17 11:54:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 11:54:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 11:54:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 11:54:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 11:54:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 11:54:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------+--------+
|    genre|genre_id|
+---------+--------+
|   Action|       1|
|    Adult|       2|
|Adventure|       3|
|Animation|       4|
|Biography|       5|
+---------+--------+
only showing top 5 rows



In [19]:
window_spec = Window.orderBy("startyear")

dim_years = movies_with_ratings.select("startyear").distinct() \
                               .withColumn("year_id", row_number().over(window_spec))
dim_years.show(5)

25/01/17 12:04:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:04:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:04:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:04:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:04:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:04:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------+-------+
|startyear|year_id|
+---------+-------+
|     1894|      1|
|     1896|      2|
|     1897|      3|
|     1898|      4|
|     1899|      5|
+---------+-------+
only showing top 5 rows



In [20]:
dim_titles = movies_with_ratings.select("tconst", "primaryTitle").distinct()
dim_titles = dim_titles.withColumnRenamed("tconst", "movie_id")
dim_titles.show(5)

+---------+--------------------+
| movie_id|        primaryTitle|
+---------+--------------------+
|tt0006481|   Die Börsenkönigin|
|tt0008687|          Tom Sawyer|
|tt0008735|The Vicar of Wake...|
|tt0009273|The Lady of the D...|
|tt0009289|      Lest We Forget|
+---------+--------------------+
only showing top 5 rows



In [21]:
fact_movies = movies_with_ratings.join(dim_genres, movies_with_ratings["genres"].contains(dim_genres["genre"]), "inner")


In [22]:
fact_movies = fact_movies.join(dim_years, "startyear", "inner")

In [26]:
fact_movies.describe()

DataFrame[summary: string, startyear: string, tconst: string, primaryTitle: string, isAdult: string, runtimeMinutes: string, genres: string, averageRating: string, numVotes: string, genre: string, genre_id: string, year_id: string]

In [27]:
fact_movies = fact_movies.select(
        col("tconst").alias("movie_id"),
        col("averageRating").alias("avg_rating"),
        col("numVotes").alias("num_votes"),
        col("runtimeMinutes").alias("runtime"),
        col("genre_id"),
        col("year_id")
)
fact_movies.show(5)

25/01/17 12:17:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:17:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:17:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:17:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:17:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:17:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 1

+---------+----------+---------+-------+--------+-------+
| movie_id|avg_rating|num_votes|runtime|genre_id|year_id|
+---------+----------+---------+-------+--------+-------+
|tt0000009|       5.4|      216|     45|      21|      1|
|tt0000147|       5.2|      544|    100|       8|      3|
|tt0000147|       5.2|      544|    100|      19|      3|
|tt0000147|       5.2|      544|    100|      23|      3|
|tt0000574|       6.0|      952|     70|       1|     12|
+---------+----------+---------+-------+--------+-------+
only showing top 5 rows



In [28]:
fact_movies.write.parquet("fact_movies.parquet")
dim_genres.write.parquet("dim_genres.parquet")
dim_years.write.parquet("dim_years.parquet")
dim_titles.write.parquet("dim_titles.parquet")

25/01/17 12:24:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:24:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:24:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:24:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:24:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 12:24:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 1

In [29]:
fact_movies.show()

25/01/17 13:23:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 13:23:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 13:23:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 13:23:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 13:23:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 13:23:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/01/17 1

+---------+----------+---------+-------+--------+-------+
| movie_id|avg_rating|num_votes|runtime|genre_id|year_id|
+---------+----------+---------+-------+--------+-------+
|tt0000009|       5.4|      216|     45|      21|      1|
|tt0000147|       5.2|      544|    100|       8|      3|
|tt0000147|       5.2|      544|    100|      19|      3|
|tt0000147|       5.2|      544|    100|      23|      3|
|tt0000574|       6.0|      952|     70|       1|     12|
|tt0000574|       6.0|      952|     70|       3|     12|
|tt0000574|       6.0|      952|     70|       5|     12|
|tt0000591|       5.7|       29|     90|       9|     13|
|tt0000615|       4.3|       27|   NULL|       9|     13|
|tt0000630|       3.1|       30|   NULL|       9|     14|
|tt0000675|       4.2|       22|   NULL|       9|     14|
|tt0000679|       5.2|       78|    120|       3|     14|
|tt0000679|       5.2|       78|    120|      11|     14|
|tt0000886|       4.5|       39|   NULL|       9|     16|
|tt0000941|   

In [31]:
spark.stop()